<font size="6"><b>Building Data Genome 2.0</b></font><br>
<font size="4"><b>All-meters datasets creation: hourly and daily timestamp</b></font><br>
<br>
<font size="2">Biam! (pic.biam@gmail.com)</font>

In [1]:
import pandas as pd
import numpy as np
from glob import glob

# Meters data

## Data join

In [2]:
path = "..\\data\\meters\\raw\\"
path1 = "..\\data\\meters\\processed\\"

In [3]:
# files in directory
files = glob(path + "*.csv")

In [4]:
files

['..\\data\\meters\\raw\\chilledwater.csv',
 '..\\data\\meters\\raw\\electricity.csv',
 '..\\data\\meters\\raw\\gas.csv',
 '..\\data\\meters\\raw\\hotwater.csv',
 '..\\data\\meters\\raw\\irrigation.csv',
 '..\\data\\meters\\raw\\solar.csv',
 '..\\data\\meters\\raw\\steam.csv',
 '..\\data\\meters\\raw\\water.csv']

In [11]:
dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("\\")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
complete_data = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

In [12]:
complete_data.head()

,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Gwyneth_Panther_Office,NaN,chilledwater
1,2016-01-01 01:00:00,Gwyneth_Panther_Office,NaN,chilledwater
2,2016-01-01 02:00:00,Gwyneth_Panther_Office,NaN,chilledwater
3,2016-01-01 03:00:00,Gwyneth_Panther_Office,NaN,chilledwater
4,2016-01-01 04:00:00,Gwyneth_Panther_Office,NaN,chilledwater


In [13]:
# Check the meters
complete_data["meter"].value_counts()

electricity     27684432
chilledwater     9736920
steam            6491280
hotwater         3245640
gas              3105288
water            2561424
irrigation        649128
solar              87720
Name: meter, dtype: int64

In [14]:
# Check number of buildings
len(np.unique(complete_data["building_id"]))

1636

In [15]:
# Rounds to 4 decimals
complete_data["meter_reading"] = round(complete_data["meter_reading"],4)

## Hourly timestamp

Meters reading are currently hourly.

In [ ]:
complete_data.to_csv(path1 + "allmeters.csv", index=False)

## Daily timestamp

Daily readings are summarized as the sum of all the readings of that day.

In [16]:
# Converts timestamp to datetime object
complete_data["timestamp"] = pd.to_datetime(complete_data["timestamp"], format='%Y-%m-%d %H:%M:%S')

In [17]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53561832 entries, 0 to 53561831
Data columns (total 4 columns):
timestamp        datetime64[ns]
building_id      object
meter_reading    float64
meter            object
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.6+ GB


In [18]:
# Sums all readings per day
complete_data = complete_data.groupby(by=["building_id","meter",complete_data['timestamp'].dt.date]).sum()

In [19]:
# Reset indexes to ungroup
complete_data = complete_data.reset_index()

In [20]:
complete_data.head()

,building_id,meter,timestamp,meter_reading
0,Aaron_Panther_Education,chilledwater,2016-01-01,0.0
1,Aaron_Panther_Education,chilledwater,2016-01-02,0.0
2,Aaron_Panther_Education,chilledwater,2016-01-03,0.0
3,Aaron_Panther_Education,chilledwater,2016-01-04,0.0
4,Aaron_Panther_Education,chilledwater,2016-01-05,0.0


In [ ]:
# export csv
complete_data.to_csv(path1 + "allmeters_daily.csv", index=False)

In [ ]:
del complete_data